# 선형 분류 모델

In [1]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
def example_plot(xy, labels, a, b, title, filename=None):
    # Shape
    c_shape = ['bs', 'r^']

    # 1. Point
    for k, (point, label) in enumerate(zip(xy, labels),1):
        x,y = point
        plt.plot(x, y, c_shape[label[0]], mec='k')
        plt.text(x, y, '$P_{}$'.format(k), size=15, \
                 verticalalignment='top', horizontalalignment='left')

    # 2. Decision line
    tmp = np.linspace(0,1,500)
    decision_line = a * tmp + b
    plt.plot(tmp, decision_line, 'k-', linewidth=3)

    # 3. Contour
    X, Y = np.meshgrid(tmp, tmp)
    Z = np.zeros_like(X)
    Z[Y > a * X + b] = 1

    plt.contourf(X, Y, Z, cmap='coolwarm')
    # plt.axis('equal')
    plt.grid(linestyle='--', alpha=0.5)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.title(title)
    if filename:
        plt.savefig(filename, dpi=300)
    plt.show()

In [3]:
def example_plot_only_line(xy, labels, a, b, title, filename=None):
    # Shape
    c_shape = ['bs', 'r^']

    # 1. Point
    for k, (point, label) in enumerate(zip(xy, labels),1):
        x,y = point
        plt.plot(x, y, c_shape[label[0]], mec='k')
        plt.text(x, y, '$P_{}$'.format(k), size=15, \
                 verticalalignment='top', horizontalalignment='left')

    # 2. Decision line
    tmp = np.linspace(0,1,500)
    decision_line = a * tmp + b
    plt.plot(tmp, decision_line, 'k-', linewidth=3)

    # 3. Contour
#     X, Y = np.meshgrid(tmp, tmp)
#     Z = np.zeros_like(X)
#     Z[Y > a * X + b] = 1

#     plt.contourf(X, Y, Z, cmap='coolwarm')
    # plt.axis('equal')
    plt.grid(linestyle='--', alpha=0.5)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.title(title)
    if filename:
        plt.savefig(filename, dpi=300)
    plt.show()

In [4]:
def example_plot_wo_contour(xy, labels, title, filename=None):
    # Shape
    c_shape = ['bs', 'r^']

    # 1. Point
    for k, (point, label) in enumerate(zip(xy, labels),1):
        x,y = point
        plt.plot(x, y, c_shape[label[0]], mec='k')
        plt.text(x, y, '$P_{}$'.format(k), size=15, \
                 verticalalignment='top', horizontalalignment='left')

#     # 2. Decision line
#     tmp = np.linspace(0,1,500)
#     decision_line = a * tmp + b
#     plt.plot(tmp, decision_line, 'k-', linewidth=3)

    # 3. Contour
#     X, Y = np.meshgrid(tmp, tmp)
#     Z = np.zeros_like(X)
#     Z[Y > a * X + b] = 1

#     plt.contourf(X, Y, Z, cmap='coolwarm')
    # plt.axis('equal')
    plt.grid(linestyle='--', alpha=0.5)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.title(title)
    if filename:
        plt.savefig(filename, dpi=300)
    plt.show()

In [5]:
def plot_model(sess, model, feature_labmda, title):
    from pandas import DataFrame
    xx, yy = np.meshgrid(np.linspace(-0.05,1.1,200), np.linspace(-0.05,1.1,200))
    prediction = sess.run(model, feed_dict={x: np.array([feature_labmda(xxval, yyval) for xxval, yyval in zip(xx.flatten(), yy.flatten())])})
    Z = prediction.reshape(xx.shape)
    df = DataFrame(dict(x=xy[:,0], y=xy[:,1], label=labels.flatten()))
    markers = {0:'bs', 1:'r^'}
    _, ax = plt.subplots(figsize=(5, 5))
    cs = ax.contourf(xx, yy, Z, 20, cmap='coolwarm', alpha=.9)
    ax.clabel(cs, colors='gray')
    cs = ax.contour(xx, yy, Z, cmap='gray', levels=[0, 0.5, 1.0], linestyles='--', linewidths=2)
    ax.clabel(cs, colors='k')
    predictions = sess.run(model, feed_dict={x: xy})
    for k, xy0 in df[['x', 'y']].iterrows():
        x0, y0 = xy0.values
        plt.plot(x0, y0, markers[labels[k][0]], mec='k')
        plt.text(x0, y0, '$P_{}$\n({:1.0f}%)'.format(k+1, 100*predictions[k][0]), size=15, \
                 verticalalignment='top', horizontalalignment='left')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    plt.grid(linestyle='--', alpha=0.5)
    plt.title(title)
    plt.show()

In [6]:
def plot_model2(sess, model, feature_labmda, title):
    from pandas import DataFrame
    xx, yy = np.meshgrid(np.linspace(-1.5,1.5, 200), np.linspace(-1.5,1.5, 200))
    prediction = sess.run(model, feed_dict={x: np.array([feature_labmda(xxval, yyval) for xxval, yyval in zip(xx.flatten(), yy.flatten())])})
    Z = prediction.reshape(xx.shape)
    df = DataFrame(dict(x=xy[:,0], y=xy[:,1], label=labels.flatten()))
    colors = {0:'red', 1:'blue'}
    markers = {0:'bs', 1:'r^'}
    _, ax = plt.subplots(figsize=(5, 5))
    ax.contourf(xx, yy, Z, 5, cmap='coolwarm', alpha=.9)
#     ax.clabel(cs, colors='k')
    cs = ax.contour(xx, yy, Z, cmap='gray', levels=[0, 0.5, 1.0], linestyles='--', linewidths=2)
    ax.clabel(cs, colors='k')
    grouped = df.groupby('label')
#     for key, group in grouped:
#         group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key], edgecolor='k')
    predictions = sess.run(model, feed_dict={x: xy})
    for k, xy0 in df[['x', 'y']].iterrows():
        x0, y0 = xy0.values
        plt.plot(x0, y0, markers[labels[k]], mec='k', alpha=0.6)
    ax.set_xlim([-1.3, 1.3])
    ax.set_ylim([-1.3, 1.3])
    plt.grid()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt.show()

In [7]:
def plot_data(xy,labels):
    from pandas import DataFrame
    df = DataFrame(dict(x=xy[:,0], y=xy[:,1], label=labels.flatten()))
    colors = {0:'red', 1:'blue'}
    markers = {0:'bs', 1:'r^'}
    _, ax = plt.subplots(figsize=(5, 5))
    for k, xy0 in df[['x', 'y']].iterrows():
        x0, y0 = xy0.values
        plt.plot(x0, y0, markers[labels[k]], mec='k', alpha=0.6)
    ax.set_xlim([-1.3, 1.3])
    ax.set_ylim([-1.3, 1.3])
    plt.grid()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [8]:
def plot_scatter_softmax(curr_W, curr_b, xy, labels):
    x1 = np.linspace(-0.1, 1.1)
    X, Y = np.meshgrid(x1,x1)
    Z1 = X * curr_W[0,0] + Y * curr_W[1,0] + curr_b[0]
    Z2 = X * curr_W[0,1] + Y * curr_W[1,1] + curr_b[1]
    Z = np.exp(Z1) / (np.exp(Z1) + np.exp(Z2))
    markers = ['bs', 'r^']
    plt.figure(figsize=(5,5))
    cs = plt.contourf(X, Y, Z, np.linspace(0, 1, 11), cmap='coolwarm')
    plt.clabel(cs, colors='k')
    for k, xy0 in enumerate(xy):
        x0, y0 = xy0
        z1 = x0 * curr_W[0,0] + y0 * curr_W[1,0] + curr_b[0]
        z2 = x0 * curr_W[0,1] + y0 * curr_W[1,1] + curr_b[1]
        z = np.array([z1,z2])
        softmax_z = np.exp(z) / np.sum(np.exp(z))
        plt.plot(x0, y0, markers[labels[k][0]], mec='k')
        plt.text(x0, y0, '$P_{}$\n({:1.1f},{:1.1f})'.format(k+1, softmax_z[0], softmax_z[1]), size=15, \
                 verticalalignment='top', horizontalalignment='left')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.show()